In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from time import sleep
from fake_useragent import UserAgent
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings 
warnings.filterwarnings('ignore')

In [21]:
user_agent = UserAgent()
school_name = []
school_link = []
for i in range(1,3):
    print("Scraping Page: ", i)
    main_url = "https://www.niche.com/k12/search/best-private-high-schools/s/new-york/?page={}".format(i)
    page = requests.get(main_url, headers={'user-agent':user_agent.chrome})
    soup = BeautifulSoup(page.content, "lxml")
    all_divs = soup.select('li.search-results__list__item h2.search-result__title')
    for div in all_divs:
        school_name.append(div.get_text())

    all_divs = soup.select("li.search-results__list__item a.search-result__link")
    for div in all_divs:
        school_link.append(div["href"])

Scraping Page:  1
Scraping Page:  2


In [37]:
information = []
for i in range(len(school_link)):
    main_url = school_link[i]
    page = requests.get(main_url, headers={'user-agent':user_agent.chrome})
    soup = BeautifulSoup(page.content, "lxml")
    all_divs = soup.select('section#about div.profile__buckets div.profile__bucket--2')
    for div in all_divs:
        information.append(div.get_text())

In [39]:
school_df = pd.DataFrame({"SchoolName":school_name,"Information":information})

In [52]:
school_df["Address"] = school_df["Information"].apply(lambda x: x.split("Address")[1]).apply(lambda x: x.split("Telephone")[0])


In [57]:
school_df["Telephone"] = school_df["Information"].apply(lambda x: x.split("Telephone")[1]).apply(lambda x: x.split("Website")[0])


In [61]:
school_df["Link"] = school_df["Information"].apply(lambda x: "https://www." + x.split("Website")[1])


In [66]:
school_df.drop("Information",axis = 1, inplace = True)

In [67]:
school_df.head()

,SchoolName,Address,Telephone,Link
0,Trinity School,"139 W 91st StNew York, NY 10024",(212) 932-6858,https://www.trinityschoolnyc.org/page
1,Regis High School,"55 E 84th StNew York, NY 10028",(212) 288-1100,https://www.regis.org
2,Horace Mann School,"231 West 246th StBronx, NY 10471",(718) 432-4000,https://www.horacemann.org
3,Collegiate School,"301 Freedom Place SouthNew York, NY 10069",(212) 812-8521,https://www.collegiateschool.org
4,Brearley School,"610 E 83rd StNew York, NY 10028",(212) 744-8582,https://www.brearley.org


In [69]:
school_df.to_excel("PrivateHighSchool.xlsx",sheet_name="PrivateHighSchool")